In [5]:
# セル 1: 必要なライブラリ読み込み
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from datetime import datetime

from segment_anything import sam_model_registry, SamPredictor


In [7]:
# セル 2: モデル読み込みと初期設定
sam_checkpoint = "../sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)


c:\aitools\segment-anything\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [9]:
# セル 3: 画像読み込み
image_path = "images/img004.jpg"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
predictor.set_image(image)


In [10]:
# セル 4: クリックで座標取得
%matplotlib tk
plt.imshow(image)
plt.title("Click a point (approx. center of target), then close the window.")
points = plt.ginput(1)
plt.close()

input_point = np.array(points, dtype=np.int32)
input_label = np.array([1])

In [11]:
# セル 5: バウンディングボックスの選択
plt.imshow(image)
plt.title("Click top-left and bottom-right corners")
box_points = plt.ginput(2)
plt.close()

x1, y1 = map(int, box_points[0])
x2, y2 = map(int, box_points[1])
input_box = np.array([x1, y1, x2, y2])

In [12]:
# セル 6: マスク生成
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box=input_box[None, :],
    multimask_output=True
)

In [21]:
# セル 7: 候補マスク表示
for i in range(len(masks)):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.imshow(masks[i], alpha=0.5)
    plt.title(f"Mask Candidate {i}")
    plt.axis("off")
    plt.show()


In [24]:
# セル 8: 最良マスクの選択（ユーザーに番号入力を促す）

# 総マスク数を表示
print(f"候補マスクの数: {len(masks)}")

# 入力を受け付けてインデックス化
while True:
    try:
        idx = int(input(f"0〜{len(masks)-1} の中から最良のマスク番号を選んでください: "))
        if 0 <= idx < len(masks):
            best_mask_index = idx
            break
        else:
            print("範囲外の数値です。もう一度入力してください。")
    except ValueError:
        print("数値を入力してください。")

# マスク適用
masked_image = image.copy()
masked_image[~masks[best_mask_index]] = 0

plt.figure(figsize=(10, 10))
plt.imshow(masked_image)
plt.title("Final extracted character")
plt.axis("off")
plt.show()


候補マスクの数: 3


0〜2 の中から最良のマスク番号を選んでください:  1


In [23]:
# セル 9: 保存処理（元画像と同名、出力先は notebooks/masked_images/）

# 出力ディレクトリ
output_dir = "masked_images"
os.makedirs(output_dir, exist_ok=True)  # フォルダがなければ作成

# 元画像と同名のファイル名（拡張子は .jpg 固定）
filename = os.path.splitext(os.path.basename(image_path))[0]
output_filename = os.path.join(output_dir, f"{filename}.jpg")

# 保存処理
cv2.imwrite(output_filename, cv2.cvtColor(masked_image, cv2.COLOR_RGB2BGR))
print("Saved to:", output_filename)


Saved to: masked_images\img004.jpg
